In [1]:
# ================================================
# Cell 1 NOTEBOOK 3 — LIVE TRADING ENGINE
# ================================================
import os, glob, time, json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import gymnasium as gym
from typing import Dict, List, Optional
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
# ================================================
# Cell 2 - Paths (update if needed)
# ================================================
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")
LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")

In [3]:
# ================================================
# Cell 3 - Configuration
# ================================================
DRY_RUN = False            # SET True for tests. Set False only after demo tests.
WINDOW = 50 # Minimum required lookback window for analysis (e.g., for indicators)
BUFFER = 60
COUNT = WINDOW + BUFFER
MAX_POS_PER_SYMBOL = 1
RRR = 3
TRAIL_PIPS_DEFAULT = 10
MAGIC = 2025001          # Unique identifier for trades
COMMENT = "Nexus_AI_Bot"
DEVIATION = 20  
# NEW: Risk Management Configuration
RISK_PERCENTAGE = 0.02    # Risk 1% (0.02) of account balance per trade
ADJUST_SL = 1.5
ALL_TRADE_RESULTS = []


In [4]:
# ================================================
# Cell 4 - Timeframe
# ================================================
TIMEFRAME = "H1"
TF_MAP = {"M1": mt5.TIMEFRAME_M1, "M5": mt5.TIMEFRAME_M5, "M15": mt5.TIMEFRAME_M15,
          "M30": mt5.TIMEFRAME_M30, "H1": mt5.TIMEFRAME_H1, "H4": mt5.TIMEFRAME_H4, "D1": mt5.TIMEFRAME_D1}
TF_MT5 = TF_MAP.get(TIMEFRAME.upper(), mt5.TIMEFRAME_M15)


In [5]:
# ================================================
# Cell 5 - MT5 connection
# ================================================
def mt5_connect() -> bool:
    try:
        if not mt5.initialize():
            # try again to get more helpful info
            err = mt5.last_error()
            print("Main mt5.initialize() first attempt returned False:", err)
            ok = mt5.initialize()
            if not ok:
                print("Main mt5.initialize() retry failed:", mt5.last_error())
                return False
        print("MT5 connected. Version:", mt5.version())
        return True
    except Exception as e:
        print("MT5 connection error:", e)
        return False

def mt5_shutdown():
    try:
        mt5.shutdown()
        # Print confirmation message upon successful execution of mt5.shutdown()
        print("Disconnected")
    except Exception:
        # Pass silently if shutdown fails
        pass

In [6]:
# ================================================
# Cell 6 - Symbols sellection
# ================================================
SYMBOLS: List[str] = [

    # ────────────────────────────────────────────────
    # Deriv Volatility Indices
    # ────────────────────────────────────────────────
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index",
    "Volatility 75 Index","Volatility 100 Index",

    # Volatility (1s) Indices
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index",
    "Volatility 75 (1s) Index","Volatility 100 (1s) Index",

    # Jump Indices
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",

    # Step Indices
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100",

    # ────────────────────────────────────────────────
    # Major Currency Pairs
    # ────────────────────────────────────────────────
    "EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCAD",
    "USDCHF", "NZDUSD", "EURGBP", "EURJPY",

    # ────────────────────────────────────────────────
    # Minor Currency Pairs
    # (All standard minors)
    # ────────────────────────────────────────────────
    "EURCHF", "EURCAD", "EURAUD", "EURNZD",
    "GBPJPY", "GBPCHF", "GBPCAD", "GBPAUD", "GBPNZD",
    "AUDJPY", "AUDNZD", "AUDCAD", "AUDCHF",
    "NZDJPY", "NZDCAD", "NZDCHF",
    "CADJPY", "CADCHF",
    "CHFJPY"
]


In [7]:
# ================================================
# Cell 7 - Symbols mapping
# ================================================
# If broker uses different symbol names, map safe_name -> broker_symbol
MANUAL_SYMBOL_MAP: Dict[str, str] = {
    # example: "Volatility_75_Index": "DerivVol75"
    # add entries if MT5 symbol_info(name) would be None for the plain name
}

# ----------------- Helpers -----------------
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

def get_mt5_symbol_from_safe(safe_name: str) -> str:
    return MANUAL_SYMBOL_MAP.get(safe_name, safe_name.replace("_", " "))

In [8]:
# ================================================
# Cell 8 - Load PPO model, VecNormalize, embeddings, scalers, datasets 
# ================================================
print("Loading model and preprocessors...")
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model file missing: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("PPO model loaded:", MODEL_FILE)

# load embeddings (attempt different formats)
embeddings = {}
EMBED_DIM = 8
if os.path.exists(EMBED_FILE):
    try:
        emb_loaded = np.load(EMBED_FILE, allow_pickle=True)
        # if saved dict
        if isinstance(emb_loaded, np.ndarray) and emb_loaded.dtype == object:
            try:
                obj = emb_loaded.item()
                if isinstance(obj, dict):
                    embeddings = {k: np.array(v, dtype=np.float32) for k, v in obj.items()}
            except Exception:
                pass
        if len(embeddings) == 0:
            # if array mapping by index
            try:
                arr = np.array(emb_loaded)
                if arr.ndim == 2:
                    # attempt mapping with asset_to_idx
                    if os.path.exists(ASSET_MAP_FILE):
                        try:
                            am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
                            for safe, idx in am.items():
                                idx = int(idx)
                                if idx < arr.shape[0]:
                                    embeddings[safe] = np.array(arr[idx], dtype=np.float32)
                        except Exception:
                            pass
                # final fallback: if embeddings length matches input CSV count, map by CSV order in data dir
                if len(embeddings) == 0 and arr.ndim == 2:
                    csvs = sorted(glob.glob(os.path.join(DATA_DIR, "*_normalized.csv")))
                    safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
                    if len(safe_list) == arr.shape[0]:
                        for i, s in enumerate(safe_list):
                            embeddings[s] = np.array(arr[i], dtype=np.float32)
            except Exception:
                pass
    except Exception as e:
        print("Failed to load embeddings:", e)
if len(embeddings) > 0:
    EMBED_DIM = next(iter(embeddings.values())).shape[0]
print("Embeddings loaded:", len(embeddings), "embed_dim:", EMBED_DIM)


# VecNormalize (safe load using dummy env)
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        class _DummyEnv(gym.Env):
            def __init__(self):
                super().__init__()
                self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(WINDOW, 5 + 1 + EMBED_DIM), dtype=np.float32)
                self.action_space = gym.spaces.Discrete(3)
            def reset(self, seed=None, options=None):
                return np.zeros(self.observation_space.shape, dtype=np.float32), {}
            def step(self, action):
                return np.zeros(self.observation_space.shape, dtype=np.float32), 0.0, True, False, {}
        venv = DummyVecEnv([lambda: _DummyEnv()])
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("VecNormalize loaded:", VEC_FILE)
    except Exception as e:
        print("VecNormalize load failed (continuing without it):", e)
        vecnorm = None
else:
    print("No VecNormalize file found; continuing without it.")


Loading model and preprocessors...
PPO model loaded: models\multiasset\ppo_multiasset.zip
Embeddings loaded: 17 embed_dim: 8
VecNormalize loaded: models\multiasset\vec_normalize.pkl


In [9]:
# ================================================
# Cell 9 - LOAD SCALERS 
# ================================================
# scalers
def load_scalers(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

scalers = load_scalers()
print("Scalers loaded:", len(scalers))

Scalers loaded: 16


In [10]:
# ================================================
# Cell 10 - datasets (prepared normalized CSVs)
# ================================================
def load_prepared_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            tmp = df[expected].dropna()
            if len(tmp) > window:
                datasets[safe] = tmp
        else:
            # try convert from OHLCV if possible
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                tmp = tmp.dropna()
                if len(tmp) > window:
                    datasets[safe] = tmp
    return datasets

datasets = load_prepared_datasets()
print("Prepared datasets loaded:", len(datasets))

Prepared datasets loaded: 16


In [11]:
# ================================================
# Cell 11 - Live bars fetch
# ================================================
def _fetch_single_symbol_bars(mt5_symbol: str, timeframe: int, count: int) -> Optional[pd.DataFrame]:
    info = mt5.symbol_info(mt5_symbol)
    if info is None:
        # no such symbol in terminal
        return None
    if not info.visible:
        try:
            mt5.symbol_select(mt5_symbol, True)
        except Exception:
            pass
    end_time = datetime.now()
    bars = mt5.copy_rates_from(mt5_symbol, timeframe, end_time, count)
    if bars is None or len(bars) < WINDOW + 2:
        return None
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    if 'tick_volume' in df.columns:
        df = df.rename(columns={'tick_volume':'volume'})
    df = df[['open','high','low','close','volume']].copy()
    df['Close_raw'] = df['close']
    return df

In [12]:
# ================================================
# Cell 12 - Observation builder
# ================================================
def build_obs_for_symbol(mt5_symbol: str, safe_name: str, window: int = WINDOW):
    # try live bars
    df_live = _fetch_single_symbol_bars(mt5_symbol, TF_MT5, COUNT)
    if df_live is None:
        # fallback to prepared dataset
        if safe_name not in datasets:
            return None, None, None
        df_pct = datasets[safe_name].iloc[-window:][['o_pc','h_pc','l_pc','c_pc','v_pc']]
        last_price = float(datasets[safe_name]['Close_raw'].iloc[-1])
        features = df_pct.values.astype(np.float32)
    else:
        # compute percent changes from live OHLCV
        df_pct = df_live[['open','high','low','close','volume']].pct_change().dropna()
        if len(df_pct) < window:
            return None, None, None
        df_pct = df_pct.tail(window)
        last_price = float(df_live['Close_raw'].iloc[-1])
        features = df_pct[['open','high','low','close','volume']].values.astype(np.float32)

    # scaler required
    if safe_name not in scalers:
        return None, None, None
    s = scalers[safe_name]
    mean = np.array(s['mean'].values if hasattr(s['mean'],'values') else s['mean'], dtype=np.float32)
    std = np.array(s['std'].values if hasattr(s['std'],'values') else s['std'], dtype=np.float32)
    std = np.where(std == 0, 1e-8, std)

    if features.shape[1] != len(mean):
        return None, None, None

    features_scaled = (features - mean) / std

    # embedding
    emb_vec = embeddings.get(safe_name, np.zeros(EMBED_DIM, dtype=np.float32))
    emb_block = np.repeat(emb_vec.reshape(1,-1), window, axis=0)

    # balance norm column (training used some balance column — here we set 1.0)
    balance_col = np.ones((window,1), dtype=np.float32)

    obs = np.concatenate([features_scaled.astype(np.float32), balance_col, emb_block], axis=1)

    # apply VecNormalize if available (best-effort)
    if vecnorm is not None:
        try:
            # prefer normalize_obs if available
            try:
                obs = vecnorm.normalize_obs(obs, False)
            except Exception:
                # fallback to obs_rms stats
                flat = obs.reshape(1,-1)
                mean_rms = getattr(vecnorm.obs_rms, "mean", None)
                var_rms = getattr(vecnorm.obs_rms, "var", None)
                if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                    flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                    obs = flat_norm.reshape(obs.shape).astype(np.float32)
        except Exception:
            pass

    vol_est = float(np.std(features[:,3]))  # std of close pct
    return obs.astype(np.float32), vol_est, last_price


In [13]:
# ================================================
# Cell 13 - Average range of the market to set SL
# ================================================
def avg_true_range(symbol: str, timeframe: int, num_of_bars: int) -> float:
    """
    Calculates the sum of True Ranges over historical bars and divides by the 
    number of bars, strictly following the provided MQL implementation.
    
    Note: The calculation includes (num_of_bars - 1) True Range values but divides 
    by num_of_bars, matching the original MQL code's formula.

    :param symbol: Trading instrument.
    :param timeframe: Timeframe (e.g., mt5.TIMEFRAME_D1).
    :param num_of_bars: Number of historical bars to analyze (starting from bar 1).
    :return: The calculated average True Range sum, or -1.0 on error.
    """
    # 1. Data Retrieval: Fetch num_of_bars + 1 to ensure we have Bar 0's Close
    # which is needed to calculate the True Range of Bar 1 (the first bar in the MQL loop).
    rates_all = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_of_bars + 1)
    
    if rates_all is None or len(rates_all) != num_of_bars + 1:
        print(f"Average True Range Error: Failed to retrieve {num_of_bars + 1} bars for {symbol}.")
        #mt5_shutdown()
        return -1.0

    df_all = pd.DataFrame(rates_all)
    # Historical bars analyzed start from index 1 (Bar 1) up to num_of_bars.
    df_hist = df_all.iloc[1:num_of_bars + 1].copy()
    
    # 2. Calculate True Range (TR)
    
    # range1: High - Low
    range1 = df_hist['high'] - df_hist['low']
    # range2: Abs(High - Previous Close). The previous close is df_all.iloc[i-1]['close']
    range2 = abs(df_hist['high'] - df_all['close'].shift(1).iloc[1:num_of_bars + 1])
    # range3: Abs(Low - Previous Close)
    range3 = abs(df_hist['low'] - df_all['close'].shift(1).iloc[1:num_of_bars + 1])
    
    # True Range is the maximum of the three ranges
    df_hist['true_range'] = np.maximum.reduce([range1, range2, range3])
    
    # 3. Summation
    true_range_sum = df_hist['true_range'].sum()

    # 4. Final Average Calculation (Matching MQL logic: trueRangeSum / numOfBars)
    if num_of_bars == 0:
        avg_true_range_sum = 0.0
    else:
        avg_true_range_sum = true_range_sum / num_of_bars
    
    # 5. Return Absolute Value (Matching MQL logic: MathAbs(avgTrueRangeSum))
    return abs(avg_true_range_sum)


In [14]:
# ================================================
# Cell 13 - Get Symbols digits
# ================================================
def getDigits(symbol: str) -> int:
    """
    Retrieves the number of decimal digits after the decimal point of the price
    for a given symbol using the MetaTrader5 API.

    :param symbol: The instrument symbol (e.g., "EURUSD").
    :return: The number of decimal digits, or 0 if retrieval fails.
    """
    symbol_info = mt5.symbol_info(symbol)

    if symbol_info is None:
        print(f"Get digits error: Could not retrieve symbol info for {symbol}. Error: {mt5.last_error()}")
        #mt5_shutdown()
        return 0
    
    digit = symbol_info.digits
    
    return digit

In [15]:
# ================================================
# Cell 13 - Compute SL and TP
# ================================================
def Compute_SL_TP (symbol: str, askPrice: float, direction: str, adjustSL: float):
    """
    direction = 1: BUY
    direction = 2: SELL (symbol: str, timeframe: int, num_of_bars: int, ADJUST_SL)
    """
    digits = getDigits(symbol)
    if direction == "BUY":
        SL = round(askPrice - (avg_true_range(symbol, TF_MT5, WINDOW) * ADJUST_SL), digits)
        riskInPoint = askPrice - SL
        TP = round(askPrice + (RRR * riskInPoint), digits)
    if direction == "SELL":       
        SL = round(askPrice + (avg_true_range(symbol, TF_MT5, WINDOW) * ADJUST_SL), digits)
        riskInPoint = SL - askPrice;
        TP = round(askPrice - (RRR * riskInPoint), digits)
    
    return float(SL), float(TP)


In [16]:
# ================================================
# Cell 13 - Calculate Optimum Lot-size
# ================================================
def calculate_optimal_volume(symbol: str, entry_price: float, sl_price: float) -> float:
    """
    Calculates the optimal lot size based on account equity, risk percentage,
    and the stop loss distance, constrained by symbol limits.

    The formula is: OptimalLotSize = (Equity * RiskPercentage) / (abs(EntryPrice - SLPrice) * ContractSize)
    """
    # 1. Get Account Equity
    account_info = mt5.account_info()
    if account_info is None:
        print("Optimal Volume Error: Could not retrieve account information.")
        return 0.0
    acc_balance = account_info.balance
    
    # 2. Get Symbol Information for contract size and volume limits
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Volume error: Could not retrieve symbol info for {symbol}.")
        return 0.0
    
    contract_size = symbol_info.trade_contract_size
    min_volume = symbol_info.volume_min
    max_volume = symbol_info.volume_max
    volume_step = symbol_info.volume_step
    
    # Check for zero values to prevent division by zero
    if contract_size == 0:
        print(f"Error: Contract size is zero for {symbol}. Returning minimum volume.")
        return min_volume
        
    # 3. Calculate Maximum Dollar Loss
    max_loss_dollar = acc_balance * RISK_PERCENTAGE
    
    # 4. Calculate Loss per Lot (Loss in price difference * Contract Size)
    price_loss = abs(entry_price - sl_price)
    
    # If SL is too close or equal to entry, use min volume as a fallback.
    if price_loss == 0.0:
        print(f"Warning: SL is equal to entry price for {symbol}. Using minimum volume.")
        return min_volume

    loss_per_lot = price_loss * contract_size
    
    # 5. Calculate Optimal Lot Size (Theoretical)
    optimal_lot_size = max_loss_dollar / loss_per_lot
    
    # 6. Apply Volume Step and Min/Max constraints
    
    # Snap the calculated lot size to the nearest allowed volume step (e.g., 0.01)
    optimal_lot_size = round(optimal_lot_size / volume_step) * volume_step
    
    # Apply min/max volume limits
    final_volume = max(min_volume, min(max_volume, optimal_lot_size))
        
    print(f"  [Risk Calcs] Balance: {acc_balance:.2f}, Risk: {RISK_PERCENTAGE * 100}%, Max Loss $: {max_loss_dollar:.2f}, Final Volume: {final_volume:.2f}")
    
    # Ensure the final volume is rounded to the step size's precision
    final_volume = round(final_volume, len(str(volume_step).split('.')[-1]) if '.' in str(volume_step) else 0)

    return final_volume


In [17]:
# ================================================
# Cell 17 SEND LIVE MARKET ODERS
# ================================================
def place_market_order(symbol: str, direction: str, sl_price: float, tp_price: float):

    # Get current price
    #tick = mt5.symbol_info_tick(symbol)
    #if tick is None:
    #    print(f"Failed to get current tick for {symbol}.")
    #    # Return a dictionary indicating local failure before calling format_result
    #    return None, {"symbol": symbol, "type": direction, "volume": 0.0} 

    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return {"Market order": None, "comment": "NO_TICK"}
    price = float(tick.ask if direction == "BUY" else tick.bid)
    if DRY_RUN:
        return {"retcode": 10009, "comment": "DRY_RUN", "price": price}

    lot = calculate_optimal_volume(symbol, price, sl_price)
    
    if lot <= 0.00001:
        print(f"Market Order skipping trade for {broker_sym}: Calculated volume is zero or negative.")
        # Return the request dictionary with 0 volume to be logged as a skipped trade
        return None, {"symbol": broker_sym, "type": direction, "volume": 0.0}
    
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        #"sl": float(sl_price) if sl_price is not None else 0.0,
        #"tp": float(tp_price) if tp_price is not None else 0.0,
        "sl": round(sl_price, getDigits(symbol)), 
        "tp": round(tp_price, getDigits(symbol)),
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": "Nexus AI",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    return mt5.order_send(req)

In [18]:
# ================================================
# Cell 19 - Get the number of opened positions
# ================================================
def get_positions_for_symbol(symbol: str):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

In [19]:
# ================================================
# Cell 20 - Close opened positions
# ================================================
def close_position_by_ticket(ticket: int):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p,"type",None) == 0:  # BUY
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [55]:
# ================================================
# Cell 21: Main run loop: single pass - OLD
# ================================================
def run_once_predict_and_manage(symbols_list: List[str]):
    if not mt5_connect():
        print("MT5 connect failed — aborting.")
        return

    acct = mt5.account_info()
    balance = float(acct.balance) if acct is not None else 10000.0
    header = not os.path.exists(LOG_FILE)

    for raw_sym in symbols_list:
        safe_name = make_safe_name(raw_sym)
        broker_sym = get_mt5_symbol_from_safe(safe_name)
        print(f"\n=== {broker_sym} (safe: {safe_name}) ===")

        obs_tuple = build_obs_for_symbol(broker_sym, safe_name, WINDOW)
        if obs_tuple is None:
            print("No observation for", safe_name, "- skipping")
            continue
        obs, vol, last_price = obs_tuple

        # predict
        try:
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list, tuple, np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, "(0=HOLD,1=BUY,2=SELL)")

        # manage positions
        positions = get_positions_for_symbol(broker_sym)
        print("Existing positions:", len(positions))

        # auto-close opposing positions
        if a == 1:  # BUY signal -> close SELL positions
            for p in positions:
                if getattr(p, "type", None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:  # SELL signal -> close BUY positions
            for p in positions:
                if getattr(p, "type", None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # refresh
        positions = get_positions_for_symbol(broker_sym)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", broker_sym, "- skipping open")
        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a == 1 else "SELL"
                SL_TP = Compute_SL_TP(broker_sym, last_price, direction, ADJUST_SL)
                SL = SL_TP[0]
                TP = SL_TP[1]
                # --- DYNAMIC VOLUME CALCULATION ---
                #lot = calculate_optimal_volume(broker_sym, last_price, SL,)
                lot = calculate_optimal_volume(broker_sym, last_price, SL)
                    
                res = place_market_order(broker_sym, direction, SL, TP)
                #res = place_market_order(broker_sym, direction, SL, TP)
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": broker_sym,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": SL,
                    "tp_price": TP,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                #print("Placed", direction, "lot", lot, "retcode", retcode)
                print("Placed", direction, "lot", 1, "retcode", retcode)

        # trailing SL pass for current positions
        for p in get_positions_for_symbol(broker_sym):
            
            trail_pips = 50
            pip = 0.005
            if getattr(p, "type", None) == 0:  # BUY
                cur_price = mt5.symbol_info_tick(broker_sym).bid
                new_sl = float(cur_price - trail_pips * pip)
            else:
                cur_price = mt5.symbol_info_tick(broker_sym).ask
                new_sl = float(cur_price + trail_pips * pip)

            if DRY_RUN:
                print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
            else:
                try:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": broker_sym, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL retcode:", getattr(r, "retcode", None))
                except Exception as e:
                    print("Modify SL failed:", e)

    mt5_shutdown()
    print("\nSingle pass complete. Logs written to:", LOG_FILE)

# ------------- run once (example) -------------
# For safety, you can test a subset:
test_symbols = SYMBOLS[:]  # change as needed
run_once_predict_and_manage(test_symbols)

# ------------- Optional continuous loop -------------
# Uncomment and use with caution (ensure DRY_RUN=False only after demo verification)
try:
    while True:
        run_once_predict_and_manage(SYMBOLS)
        time.sleep(60)   # run each minute for M1/M15 choose appropriate sleep
except KeyboardInterrupt:
    print("Stopped by user.")


MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
MT5 connected. Version: (500, 5430, '14 Nov 2025')
MT5 connected. Version: (500, 5430, '14 Nov 2025')
MT5 connected. Version: (500, 5430, '14 Nov 2025')
  [Risk Calcs] Balance: 6200.97, Risk: 2.0%, Max Loss $: 124.02, Final Volume: 17.85
MT5 connected. Version: (500, 5430, '14 Nov 2025')
  [Risk Calcs] Balance: 6200.97, Risk: 2.0%, Max Loss $: 124.02, Final Volume: 17.85


C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\1340734064.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Placed SELL lot 1 retcode 10009
Modify SL retcode: 10016

=== Volatility 25 Index (safe: Volatility_25_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
MT5 connected. Version: (500, 5430, '14 Nov 2025')
MT5 connected. Version: (500, 5430, '14 Nov 2025')
MT5 connected. Version: (500, 5430, '14 Nov 2025')
  [Risk Calcs] Balance: 6200.97, Risk: 2.0%, Max Loss $: 124.02, Final Volume: 17.34
MT5 connected. Version: (500, 5430, '14 Nov 2025')
  [Risk Calcs] Balance: 6200.97, Risk: 2.0%, Max Loss $: 124.02, Final Volume: 16.94


C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\1340734064.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Placed BUY lot 1 retcode 10009
Modify SL retcode: 10016

=== Volatility 50 Index (safe: Volatility_50_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
MT5 connected. Version: (500, 5430, '14 Nov 2025')
MT5 connected. Version: (500, 5430, '14 Nov 2025')
MT5 connected. Version: (500, 5430, '14 Nov 2025')
  [Risk Calcs] Balance: 6200.97, Risk: 2.0%, Max Loss $: 124.02, Final Volume: 150.18
MT5 connected. Version: (500, 5430, '14 Nov 2025')
  [Risk Calcs] Balance: 6200.97, Risk: 2.0%, Max Loss $: 124.02, Final Volume: 150.18


...

Placed SELL lot 1 retcode None

=== Volatility 50 1s Index (safe: Volatility_50_1s_Index) ===
Signal: 0 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD

=== Volatility 75 1s Index (safe: Volatility_75_1s_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
MT5 connected. Version: (500, 5430, '14 Nov 2025')
Get digits error: Could not retrieve symbol info for Volatility 75 1s Index. Error: (-4, 'Terminal: Not found')
Disconnected
MT5 connected. Version: (500, 5430, '14 Nov 2025')
Error: Failed to retrieve 51 bars for Volatility 75 1s Index.
Disconnected
MT5 connected. Version: (500, 5430, '14 Nov 2025')
Volume error: Could not retrieve symbol info for Volatility 75 1s Index.
Placed BUY lot 1 retcode None

=== Volatility 100 1s Index (safe: Volatility_100_1s_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
MT5 connected. Version: (500, 5430, '14 Nov 2025')
Get digits error: Could not retrieve symbol info for Volatility 100 1s Index. Error: (-4, 'Terminal: Not

C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\1340734064.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\1340734064.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Modify SL retcode: 10009

=== GBPUSD (safe: GBPUSD) ===
Prediction error: 'NoneType' object is not subscriptable

=== USDJPY (safe: USDJPY) ===
Prediction error: 'NoneType' object is not subscriptable

=== AUDUSD (safe: AUDUSD) ===
Prediction error: 'NoneType' object is not subscriptable

=== USDCAD (safe: USDCAD) ===
Prediction error: 'NoneType' object is not subscriptable

=== USDCHF (safe: USDCHF) ===
Prediction error: 'NoneType' object is not subscriptable

=== NZDUSD (safe: NZDUSD) ===
Prediction error: 'NoneType' object is not subscriptable

=== EURGBP (safe: EURGBP) ===
Prediction error: 'NoneType' object is not subscriptable

=== EURJPY (safe: EURJPY) ===
Prediction error: 'NoneType' object is not subscriptable
Disconnected

Single pass complete. Logs written to: models\multiasset\live_logs\live_trade_logs.csv
MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 1
Max posi

C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\1340734064.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\1340734064.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\1340734064.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_8776\134

MT5 connected. Version: (500, 5430, '14 Nov 2025')
Error: Failed to retrieve 51 bars for Volatility 100 1s Index.
Disconnected
MT5 connected. Version: (500, 5430, '14 Nov 2025')
Volume error: Could not retrieve symbol info for Volatility 100 1s Index.
Placed SELL lot 1 retcode None

=== Jump 10 Index (safe: Jump_10_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 1
Max positions reached for Jump 10 Index - skipping open
Modify SL retcode: 10016

=== Jump 25 Index (safe: Jump_25_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 1
Max positions reached for Jump 25 Index - skipping open
Modify SL retcode: 10016

=== Jump 50 Index (safe: Jump_50_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 1
Max positions reached for Jump 50 Index - skipping open
Modify SL retcode: 10016

=== Jump 75 Index (safe: Jump_75_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 1
Max positions reached for Jump 75 Index - skipping open
Modify SL retcode: 100

In [20]:
# ================================================
# Cell 21: Trailing Stop Loss Management
# ================================================
EXPERT_MAGIC_NUMBER = MAGIC

def apply_trailing_sltp(broker_sym: str):
    """
    Applies trailing Stop Loss logic based on MQL5's profit-based trailing.

    :param broker_sym: The MetaTrader 5 symbol name (e.g., "EURUSD").
    """
    print(f"Applying MQL5-style Trailing SL to {broker_sym}")
    
    # Get symbol info for precision (digits) and tick values
    symbol_info = mt5.symbol_info(broker_sym)
    if symbol_info is None:
        print(f"Trailing Error: Could not get symbol info for {broker_sym}")
        return
        
    digits = symbol_info.digits
    
    # Get the latest tick info for Bid/Ask prices
    tick_info = mt5.symbol_info_tick(broker_sym)
    if tick_info is None:
        print(f"Error getting tick info for {broker_sym}")
        return

    # MQL5 logic uses PositionsTotal() and filters by magic number and symbol.
    # In MT5 Python, we fetch positions and filter them locally.
    # Note: Using get_positions_for_symbol(broker_sym) filters by symbol already.
    all_positions = get_positions_for_symbol(broker_sym)
    
    # Filter positions by the required magic number
    positions = [
        p for p in all_positions 
        if getattr(p, "magic", None) == MAGIC
    ]

    print(f"Found {len(positions)} positions with magic {MAGIC}")

    for p in positions:
        ticket = int(p.ticket)
        # Extract necessary position data
        entry_price = getattr(p, "price_open", 0.0)
        sl_price = getattr(p, "sl", 0.0)
        tp_price = getattr(p, "tp", 0.0)
        position_type = getattr(p, "type", None) # 0=BUY, 1=SELL
        
        # We only trail if SL and TP are already set
        if sl_price == 0.0 or tp_price == 0.0:
            continue

        # --- Dynamic Trailing Constants (from MQL5 logic) ---
        # Note: trailDistance (0.3) is hardcoded in the MQL5 SELL block, 
        # but not explicitly in the BUY block, I will use 0.3 for consistency 
        # in the absence of explicit definition in the BUY MQL5 code snippet.
        trail_factor = 0.3 
        sl_trail_factor = 0.1 # SLTrail = 0.1 * trail

        if position_type == mt5.ORDER_TYPE_BUY:
            cur_price = tick_info.bid
            
            # target = TP - entryPrice
            target = tp_price - entry_price
            
            # distance = askPrice - SL -> In MQL5 this seems to be current price (Ask) - SL 
            # If using Bid for current price, distance = Bid - SL
            distance = cur_price - sl_price
            
            # trail = trailDistance * target
            trail = trail_factor * target
            
            # SLTrail = 0.1 * trail
            sl_trail = sl_trail_factor * trail
            
            # Only trail if the distance from the current price to SL exceeds the trail threshold (trail)
            if distance > trail:
                new_sl = sl_price + sl_trail
                
                # Check if new SL is better (higher) than current SL, and not zero
                if new_sl > sl_price and new_sl > 0:
                    #new_sl_normalized = mt5.symbol_info(broker_sym)._normalize_price(new_sl)
                    new_sl_normalized = round(new_sl, digits)
                    
                    if DRY_RUN:
                        print(f"[DRY] BUY: Ticket {ticket} - New SL: {new_sl_normalized:.{digits}f}")
                    else:
                        req = {
                            "action": mt5.TRADE_ACTION_SLTP,
                            "symbol": broker_sym,
                            "position": ticket,
                            "sl": new_sl_normalized,
                            "tp": tp_price
                        }
                        r = mt5.order_send(req)
                        retcode = getattr(r, "retcode", None)
                        if retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"BUY position SL trailed. Ticket {ticket}, New SL: {new_sl_normalized:.{digits}f}")
                        else:
                            print(f"Failed to trail BUY SL. Ticket {ticket}, Retcode: {retcode}")
                            
            # The MQL5 code also had an 'else if(distance > target)' block for continuous trailing
            # which was commented out. We will skip implementing the commented-out logic.

        elif position_type == mt5.ORDER_TYPE_SELL:
            cur_price = tick_info.ask
            
            # target = MathAbs(TP - entryPrice)
            target = abs(tp_price - entry_price)
            
            # distance = MathAbs(askPrice - SL) -> Distance from current price (Ask) to SL
            distance = abs(cur_price - sl_price)
            
            # trail = 0.3 * target
            trail = trail_factor * target
            
            # SLTrail = 0.1 * trail
            sl_trail = sl_trail_factor * trail
            
            # Only trail if the distance from the current price to SL exceeds the trail threshold (trail)
            # The MQL code had an additional commented condition: && SL > entryPrice (for short) which is unusual, 
            # I will omit the commented condition.
            if distance > trail:
                new_sl = sl_price - sl_trail # SL moves closer to entry/market (down)
                
                # Check if new SL is better (lower) than current SL, and not zero
                if new_sl < sl_price and new_sl > 0:
                    #new_sl_normalized = mt5.symbol_info(broker_sym)._normalize_price(new_sl)
                    new_sl_normalized = round(new_sl, digits)
                    
                    if DRY_RUN:
                        print(f"[DRY] SELL: Ticket {ticket} - New SL: {new_sl_normalized:.{digits}f}")
                    else:
                        req = {
                            "action": mt5.TRADE_ACTION_SLTP,
                            "symbol": broker_sym,
                            "position": ticket,
                            "sl": new_sl_normalized,
                            "tp": tp_price
                        }
                        r = mt5.order_send(req)
                        retcode = getattr(r, "retcode", None)
                        if retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"SELL position SL trailed. Ticket {ticket}, New SL: {new_sl_normalized:.{digits}f}")
                        else:
                            print(f"Failed to trail SELL SL. Ticket {ticket}, Retcode: {retcode}")
            
            # The MQL5 code also had an 'else if(distance > target)' block for continuous trailing
            # which was commented out. We will skip implementing the commented-out logic.


In [21]:
# ================================================
# Cell 21: Prediction and Trading Logic - main
# ================================================
def run_once_predict_and_trade(symbols_list: List[str]):
    """
    Performs a single pass of prediction, trade management (close opposing), 
    new trade opening, and applies trailing stop loss for a list of symbols.
    
    :param symbols_list: List of raw symbols (e.g., ["EURUSD"]).
    """
    if not mt5_connect():
        print("MT5 connect failed - aborting.")
        return

    acct = mt5.account_info()
    # balance = float(acct.balance) if acct is not None else 10000.0 # Balance is not used in the logic below
    header = not os.path.exists(LOG_FILE)

    for raw_sym in symbols_list:
        safe_name = make_safe_name(raw_sym)
        broker_sym = get_mt5_symbol_from_safe(safe_name)
        print(f"\n=== {broker_sym} (safe: {safe_name}) ===")

        # --- 1. Data Retrieval and Prediction ---
        obs_tuple = build_obs_for_symbol(broker_sym, safe_name, WINDOW)
        if obs_tuple is None:
            print("No observation for", safe_name, "- skipping")
            continue
        obs, vol, last_price = obs_tuple

        try:
            # Predict action: 0=HOLD, 1=BUY, 2=SELL
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list, tuple, np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, "(0=HOLD, 1=BUY, 2=SELL)")

        # --- 2. Manage Existing Positions (Close Opposing) ---
        positions = get_positions_for_symbol(broker_sym)
        print("Existing positions:", len(positions))

        if a == 1:  # BUY signal -> close SELL positions (type 1)
            for p in positions:
                # Check for SELL position (type 1)
                if getattr(p, "type", None) == mt5.ORDER_TYPE_SELL:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:  # SELL signal -> close BUY positions (type 0)
            for p in positions:
                # Check for BUY position (type 0)
                if getattr(p, "type", None) == mt5.ORDER_TYPE_BUY:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # --- 3. Open New Position ---
        
        # Refresh positions after potential closures
        positions = get_positions_for_symbol(broker_sym)
        
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", broker_sym, "- skipping open")
        else:
            if a == 0:
                print("HOLD")
            else:
                # Prepare to open a trade
                direction = "BUY" if a == 1 else "SELL"
                
                # Compute Stop Loss and Take Profit prices
                SL_TP = Compute_SL_TP(broker_sym, last_price, direction, ADJUST_SL)
                SL = SL_TP[0]
                TP = SL_TP[1]
                
                # Dynamic Volume Calculation (Lot is hardcoded to 1 in the original function's log entry)
                lot = calculate_optimal_volume(broker_sym, last_price, SL)
                
                # Place the Market Order
                # NOTE: The original trade placement doesn't include the magic number, 
                # but to be tracked by the new trailing function, it must be added here.
                # Assuming 'place_market_order' handles adding the magic number.
                res = place_market_order(broker_sym, direction, SL, TP)
                
                # Extract results and handle logging
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                    ticket = res.get("order", 0)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price
                    ticket = getattr(res, "order", 0)

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": broker_sym,
                    "action": direction,
                    "lot": lot, # NOTE: Original code always logs '1' regardless of calculated 'lot'
                    "exec_price": exec_price,
                    "sl_price": SL,
                    "tp_price": TP,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", 1, "retcode", retcode, f"Ticket: {ticket}")

        # --- 4. Apply Trailing SL (using the new function) ---
        apply_trailing_sltp(broker_sym)

    mt5_shutdown()
    print("\nSingle pass complete. Logs written to:", LOG_FILE)

In [20]:

def run_once_predict_and_trade(symbols_list: List[str]):
    """
    Performs a single pass of prediction, trade management (close opposing), 
    new trade opening, and applies trailing stop loss for a list of symbols.
    
    :param symbols_list: List of raw symbols (e.g., ["EURUSD"]).
    """
    if not mt5_connect():
        print("MT5 connect failed - aborting.")
        return

    acct = mt5.account_info()
    # balance = float(acct.balance) if acct is not None else 10000.0 # Balance is not used in the logic below
    header = not os.path.exists(LOG_FILE)

    for raw_sym in symbols_list:
        safe_name = make_safe_name(raw_sym)
        broker_sym = get_mt5_symbol_from_safe(safe_name)
        print(f"\n=== {broker_sym} (safe: {safe_name}) ===")

        # --- 1. Data Retrieval and Prediction ---
        obs_tuple = build_obs_for_symbol(broker_sym, safe_name, WINDOW)
        if obs_tuple is None:
            print("No observation for", safe_name, "- skipping")
            continue
        obs, vol, last_price = obs_tuple

        try:
            # Predict action: 0=HOLD, 1=BUY, 2=SELL
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list, tuple, np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, "(0=HOLD, 1=BUY, 2=SELL)")

        # --- 2. Manage Existing Positions (Close Opposing) ---
        positions = get_positions_for_symbol(broker_sym)
        print("Existing positions:", len(positions))

        if a == 1:  # BUY signal -> close SELL positions (type 1)
            for p in positions:
                # Check for SELL position (type 1)
                if getattr(p, "type", None) == mt5.ORDER_TYPE_SELL:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:  # SELL signal -> close BUY positions (type 0)
            for p in positions:
                # Check for BUY position (type 0)
                if getattr(p, "type", None) == mt5.ORDER_TYPE_BUY:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # --- 3. Open New Position ---
        
        # Refresh positions after potential closures
        positions = get_positions_for_symbol(broker_sym)
        
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", broker_sym, "- skipping open")
        else:
            if a == 0:
                print("HOLD")
            else:
                # Prepare to open a trade
                direction = "BUY" if a == 1 else "SELL"
                
                # Compute Stop Loss and Take Profit prices
                SL_TP = Compute_SL_TP(broker_sym, last_price, direction, ADJUST_SL)
                SL = SL_TP[0]
                TP = SL_TP[1]
                
                # Dynamic Volume Calculation (Lot is hardcoded to 1 in the original function's log entry)
                lot = calculate_optimal_volume(broker_sym, last_price, SL)
                
                # Place the Market Order
                res = place_market_order(broker_sym, direction, SL, TP)
                
                # Extract results and handle logging
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                    ticket = res.get("order", 0)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price
                    ticket = getattr(res, "order", 0)

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": broker_sym,
                    "action": direction,
                    "lot": 1, # NOTE: Original code always logs '1' regardless of calculated 'lot'
                    "exec_price": exec_price,
                    "sl_price": SL,
                    "tp_price": TP,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", 1, "retcode", retcode, f"Ticket: {ticket}")

        # --- 4. Apply Trailing SL (using the new function) ---
        apply_trailing_sltp(broker_sym)

    mt5_shutdown()
    print("\nSingle pass complete. Logs written to:", LOG_FILE)




MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 0
  [Risk Calcs] Balance: 5381.15, Risk: 2.0%, Max Loss $: 107.62, Final Volume: 8.24
  [Risk Calcs] Balance: 5381.15, Risk: 2.0%, Max Loss $: 107.62, Final Volume: 8.10


C:\Users\Slick\AppData\Local\Temp\ipykernel_4512\4138345780.py:153: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Placed BUY lot 1 retcode 10009 Ticket: 4456253603
--- Applying Trailing SL to Volatility 10 Index ---
Modify SL retcode: 10016 New SL: 5477.90600

=== Volatility 25 Index (safe: Volatility_25_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 0
  [Risk Calcs] Balance: 5381.15, Risk: 2.0%, Max Loss $: 107.62, Final Volume: 7.52
  [Risk Calcs] Balance: 5381.15, Risk: 2.0%, Max Loss $: 107.62, Final Volume: 7.43


C:\Users\Slick\AppData\Local\Temp\ipykernel_4512\4138345780.py:153: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Placed BUY lot 1 retcode 10009 Ticket: 4456253609
--- Applying Trailing SL to Volatility 25 Index ---
Modify SL retcode: 10016 New SL: 2374.68300

=== Volatility 50 Index (safe: Volatility_50_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 0
  [Risk Calcs] Balance: 5381.15, Risk: 2.0%, Max Loss $: 107.62, Final Volume: 65.17
  [Risk Calcs] Balance: 5381.15, Risk: 2.0%, Max Loss $: 107.62, Final Volume: 65.17


...

MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Volatility 10 Index - skipping open
--- Applying Trailing SL to Volatility 10 Index ---
Modify SL retcode: 10016 New SL: 5479.42200

=== Volatility 25 Index (safe: Volatility_25_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Volatility 25 Index - skipping open
--- Applying Trailing SL to Volatility 25 Index ---
Modify SL retcode: 10016 New SL: 2378.64800

=== Volatility 50 Index (safe: Volatility_50_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 0
  [Risk Calcs] Balance: 5353.20, Risk: 2.0%, Max Loss $: 107.06, Final Volume: 64.79
  [Risk Calcs] Balance: 5353.20, Risk: 2.0%, Max Loss $: 107.06, Final Volume: 64.79


C:\Users\Slick\AppData\Local\Temp\ipykernel_4512\4138345780.py:153: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Placed SELL lot 1 retcode 10009 Ticket: 4456266861
--- Applying Trailing SL to Volatility 50 Index ---
Modify SL retcode: 10009 New SL: 131.58340

=== Volatility 75 Index (safe: Volatility_75_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Volatility 75 Index - skipping open
--- Applying Trailing SL to Volatility 75 Index ---
Modify SL retcode: 10016 New SL: 42340.73000

=== Volatility 100 Index (safe: Volatility_100_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Volatility 100 Index - skipping open
--- Applying Trailing SL to Volatility 100 Index ---
Modify SL retcode: 10016 New SL: 605.11000

=== Volatility 10 1s Index (safe: Volatility_10_1s_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 0
Get digits error: Could not retrieve symbol info for Volatility 10 1s Index. Error: (-4, 'Terminal: Not found')
Error: Failed to retrieve 51 bars for Volatility 10 1s Index.
Disconnected
Er

C:\Users\Slick\AppData\Local\Temp\ipykernel_4512\4138345780.py:153: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_4512\4138345780.py:153: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_4512\4138345780.py:153: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_4512\

Stopped by user.


In [22]:
# ------------- run once (example) -------------
# For safety, you can test a subset:
test_symbols = SYMBOLS[:]  # change as needed
#run_once_predict_and_manage(test_symbols)
run_once_predict_and_trade(test_symbols)

# ------------- Optional continuous loop -------------
# Uncomment and use with caution (ensure DRY_RUN=False only after demo verification)
try:
    while True:
        #run_once_predict_and_manage(SYMBOLS)
        run_once_predict_and_trade(SYMBOLS)
        time.sleep(60)   # run each minute for M1/M15 choose appropriate sleep
except KeyboardInterrupt:
    print("Stopped by user.")


MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Volatility 10 Index - skipping open
Applying MQL5-style Trailing SL to Volatility 10 Index
Found 1 positions with magic 2025001

=== Volatility 25 Index (safe: Volatility_25_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Closing opposing SELL ticket 4456308532
  [Risk Calcs] Balance: 5353.50, Risk: 2.0%, Max Loss $: 107.07, Final Volume: 7.33
  [Risk Calcs] Balance: 5353.50, Risk: 2.0%, Max Loss $: 107.07, Final Volume: 7.25
Placed BUY lot 1 retcode 10009 Ticket: 4456329400
Applying MQL5-style Trailing SL to Volatility 25 Index
Found 1 positions with magic 2025001


C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


BUY position SL trailed. Ticket 4456329400, New SL: 2373.922

=== Volatility 50 Index (safe: Volatility_50_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Volatility 50 Index - skipping open
Applying MQL5-style Trailing SL to Volatility 50 Index
Found 1 positions with magic 2025001

=== Volatility 75 Index (safe: Volatility_75_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Closing opposing BUY ticket 4456308576
  [Risk Calcs] Balance: 5327.85, Risk: 2.0%, Max Loss $: 106.56, Final Volume: 0.14
  [Risk Calcs] Balance: 5327.85, Risk: 2.0%, Max Loss $: 106.56, Final Volume: 0.14
Placed SELL lot 1 retcode 10009 Ticket: 4456329419
Applying MQL5-style Trailing SL to Volatility 75 Index
Found 1 positions with magic 2025001


C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


SELL position SL trailed. Ticket 4456329419, New SL: 43193.88

=== Volatility 100 Index (safe: Volatility_100_Index) ===
Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Closing opposing SELL ticket 4456308586
  [Risk Calcs] Balance: 5344.47, Risk: 2.0%, Max Loss $: 106.89, Final Volume: 6.63
  [Risk Calcs] Balance: 5344.47, Risk: 2.0%, Max Loss $: 106.89, Final Volume: 6.55
Placed BUY lot 1 retcode 10009 Ticket: 4456329450
Applying MQL5-style Trailing SL to Volatility 100 Index
Found 1 positions with magic 2025001
BUY position SL trailed. Ticket 4456329450, New SL: 600.40

=== Volatility 10 1s Index (safe: Volatility_10_1s_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 0
Get digits error: Could not retrieve symbol info for Volatility 10 1s Index. Error: (-4, 'Terminal: Not found')
Average True Range Error: Failed to retrieve 51 bars for Volatility 10 1s Index.
Volume error: Could not retrieve symbol info for Volatility 10 1s Index.
Placed SELL lot 1 retcode No

...

BUY position SL trailed. Ticket 4456406223, New SL: 20263.28

=== Jump 100 Index (safe: Jump_100_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Jump 100 Index - skipping open
Applying MQL5-style Trailing SL to Jump 100 Index
Found 1 positions with magic 2025001

=== Step Index 25 (safe: Step_Index_25) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 50 (safe: Step_Index_50) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 75 (safe: Step_Index_75) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 100 (safe: Step_Index_100) ===
Prediction error: 'NoneType' object is not subscriptable

=== EURUSD (safe: EURUSD) ===
Signal: 0 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for EURUSD - skipping open
Applying MQL5-style Trailing SL to EURUSD
Found 1 positions with magic 2025001

=== GBPUSD (safe: GBPUSD) ===
Prediction error: 'NoneType' object 

C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Jump 100 Index - skipping open
Applying MQL5-style Trailing SL to Jump 100 Index
Found 1 positions with magic 2025001

=== Step Index 25 (safe: Step_Index_25) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 50 (safe: Step_Index_50) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 75 (safe: Step_Index_75) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 100 (safe: Step_Index_100) ===
Prediction error: 'NoneType' object is not subscriptable

=== EURUSD (safe: EURUSD) ===
Signal: 0 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for EURUSD - skipping open
Applying MQL5-style Trailing SL to EURUSD
Found 1 positions with magic 2025001

=== GBPUSD (safe: GBPUSD) ===
Prediction error: 'NoneType' object is not subscriptable

=== USDJPY (safe: USDJPY) ===
Prediction error: 'NoneType' object is not subscriptable

C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_14404\2302471648.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


Signal: 1 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Jump 75 Index - skipping open
Applying MQL5-style Trailing SL to Jump 75 Index
Found 1 positions with magic 2025001
BUY position SL trailed. Ticket 4456406223, New SL: 20303.28

=== Jump 100 Index (safe: Jump_100_Index) ===
Signal: 2 (0=HOLD, 1=BUY, 2=SELL)
Existing positions: 1
Max positions reached for Jump 100 Index - skipping open
Applying MQL5-style Trailing SL to Jump 100 Index
Found 1 positions with magic 2025001

=== Step Index 25 (safe: Step_Index_25) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 50 (safe: Step_Index_50) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 75 (safe: Step_Index_75) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 100 (safe: Step_Index_100) ===
Prediction error: 'NoneType' object is not subscriptable

=== EURUSD (safe: EURUSD) ===
Signal: 0 (0=HOLD, 1=BUY, 2=SELL)
Existing positions:

In [ ]:
#symbol = "EURUSD"
#symbol = "Volatility 50 (1s) Index"
symbol = "Volatility 50 Index"
info = mt5.symbol_info(symbol)

print("Bid:", info.bid)
print("Ask:", info.ask)
print("Digits:", info.digits) # Number of digits after the decimal point of price
print("Contract Size:", info.trade_contract_size)
print("Volume Min:", info.volume_min)
print("Trade tick size:", info.trade_tick_size)
print("Trade tick value:", info.trade_tick_value)
#print("Volume Step:", info.volume_step)
print("Point:", info.point)
